# Exploring Data: The City of Chicago 

In this Python notebook, we will explore 3 different datasets in order to develop a deeper understanding of the City of Chicago.</br></br>
These include:
- Socioeconomic Indicators in Chicago
- Chicago Publis Schools
- Chicago Crime Data


#### Establishing our connection.

In [1]:
import csv, sqlite3, pandas
con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

In [2]:
%load_ext sql
%sql sqlite:///FinalDB.db

#### Using Pandas to read our CSV file and push data set into database.

In [3]:
socioeconomic_df = pandas.read_csv("ChicagoCensusData.csv")
crime_df = pandas.read_csv("ChicagoCrimeData.csv")
schools_df = pandas.read_csv("ChicagoPublicSchools.csv")

In [4]:
socioeconomic_df.to_sql("CENSUS_DATA",con, if_exists='replace', index=False, method="multi")

78

In [5]:
crime_df.to_sql("CHICAGO_CRIME_DATA",con, if_exists='replace', index=False, method="multi")

533

In [6]:
schools_df.to_sql("CHICAGO_PUBLIC_SCHOOLS",con, if_exists='replace', index=False, method="multi")

566

#### Checking our metadata to confirm our tables have been created.

In [7]:
%sql select name from sqlite_master where type='table'

 * sqlite:///FinalDB.db
Done.


name
CENSUS_DATA
CHICAGO_CRIME_DATA
CHICAGO_PUBLIC_SCHOOLS


### Now let's do some exploring!

#### 1. Let's find the total number of crimes recored in the CRIME table.

In [12]:
%sql select count(*) as 'Total_Crimes'from CHICAGO_CRIME_DATA 

 * sqlite:///FinalDB.db
Done.


Total_Crimes
533


#### 2. Let's get a list of all the community areas with per capita income less than 11000.

In [14]:
%sql select COMMUNITY_AREA_NAME from CENSUS_DATA where PER_CAPITA_INCOME<11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


#### 3. Let's get a list of all the crimes involving minors. 
(Children are not considered minors for the purpose of crime analysis)

In [17]:
%sql select distinct CASE_NUMBER from CHICAGO_CRIME_DATA where DESCRIPTION like '%MINOR%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


#### 4. Let's get a list of all kidnapping crimes involving a child.

In [23]:
%sql select distinct * from CHICAGO_CRIME_DATA where PRIMARY_TYPE='KIDNAPPING' and DESCRIPTION like '%CHILD%'

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


#### 5. What kinds of crimes were conducted at schools?

##### %sql select distinct PRIMARY_TYPE from CHICAGO_CRIME_DATA where LOCATION_DESCRIPTION like '%SCHOOL%'

#### 6. Let's get a list of the average safety score for each type of school?

In [37]:
%sql select "Elementary, Middle, or High School", AVG(SAFETY_SCORE) AVERAGE_SAFETY_SCORE \
from CHICAGO_PUBLIC_SCHOOLS group by "Elementary, Middle, or High School";

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",AVERAGE_SAFETY_SCORE
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


#### 7. List 5 community areas with the highest % of households below poverty line.

In [57]:
%sql select COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY from CENSUS_DATA order by PERCENT_HOUSEHOLDS_BELOW_POVERTY desc limit 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,Riverdale,56.5
37.0,Fuller Park,51.2
68.0,Englewood,46.6
29.0,North Lawndale,43.1
27.0,East Garfield Park,42.4


#### 8. Which community area is most crime prone?

In [65]:
%%sql
select COMMUNITY_AREA_NUMBER, count(COMMUNITY_AREA_NUMBER) as Frequency 
from CHICAGO_CRIME_DATA 
group by COMMUNITY_AREA_NUMBER
order by count(COMMUNITY_AREA_NUMBER) desc limit 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,Frequency
25.0,43


#### 9. Let's find the name of the community area with the highest hardship index using sub-queries.

In [56]:
%%sql 
select COMMUNITY_AREA_NAME from CENSUS_DATA where HARDSHIP_INDEX=(select max(HARDSHIP_INDEX) from CENSUS_DATA)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


#### 10. Lastly, let's find the name of the community area with the most number of crimes using sub-queries.

In [67]:
%%sql
select COMMUNITY_AREA_NAME from CENSUS_DATA 
where COMMUNITY_AREA_NUMBER=(select COMMUNITY_AREA_NUMBER
    from CHICAGO_CRIME_DATA 
    group by COMMUNITY_AREA_NUMBER
    order by count(COMMUNITY_AREA_NUMBER) desc limit 1);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
